In [1]:
"""This cell generates synthetic data as described in Section 6.1 of the paper.
The correlation is exponentially decaying (i.e., Setting 1) and the nonzero coefficients
are all set to 1.
"""

import numpy as np
import scipy as sc
import math
from bnb import gen_synthetic


# Set the data generation parameters.
rho = 0.3 # Correlation parameter.
n = 1000 # Number of observations.
p = 100 # Number of features. 
k0 = 5 # Number of nonzero groups.
SNR = 10 # Signal-to-noise ratio.
Gsize = 10 # Group size.

x, y, group_indices, true_coefs = gen_synthetic(rho,n,p,k0,SNR,Gsize)

In [2]:
"""This cell demonstrates how to run the BnB algorithm.

The algorithm optimizes the following objective function:

    0.5*||y-XB||^2 + lambda_0 * G(B) + lambda_2 ||B||^2
    
where G(B) counts the number of nonzero groups (as defined in the paper).

"""
from bnb import BNBTree

"""
First, we create a BnBTree object and initialize it with the following:
   x: Data matrix
   y: Response vector
   group_indices: A list of size num_groups. The ith element contains the indices of the predictors in group i.
"""

tree = BNBTree(x, y, group_indices=group_indices)

"""
Call tree.solve(...) to run BnB for a given set of reguralization parameters. tree.solve accepts:
    lambda_0: The regularization parameter for the L0 norm.
    lambda_2: The regularization parameter for the L2 norm.
    m: The value of the Big M.
    warm_start: The initial solution.
"""

solver_output = tree.solve(lambda_0=10000, lambda_2=1000, m=3, warm_start=true_coefs)

Academic license - for non-commercial use only - expires 2021-04-26
Using license file /Users/hh/gurobi.lic


In [3]:
print(solver_output)
# solver_output.beta contains the solution.

Solution(cost=122611.99262027604, beta=array([0.82945267, 0.89144701, 0.91953546, 0.86141272, 0.86516223,
       0.83967938, 0.85712663, 0.91397311, 0.82921268, 0.83374505,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.85116191, 0.87063195, 0.78903885, 0.80825977, 0.81169689,
       0.85553931, 0.74675512, 0.85250995, 0.80827098, 0.82519644,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.85968506, 0.92415876, 0.92362982, 0.90563082, 0.93315325,
       0.93518654, 0.87156978, 0.89851305, 0.84637144, 0.86678905,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.84610101, 0.81319322, 0.85054379, 0.86686436, 0.80075785,
       0.76823883, 0.84076327, 0.84500014, 0.8642548 , 0.86361795,
       0.        , 0.  

In [4]:
"""This cell demonstrates how to run the CD algorithm."""

from cd import cd

# The Params dictionary specifies some internal parameters used by CD.
Params = {
    'NumIters': 1000, # maximum number of CD full cycles.
    'Tolerance': 1e-7 , # convergence tolerance.
    'Logging': False, # prints output while running
    'FeaturesOrder': 'Cyclic', # order in which CD updates the groups.
}

# x, y, group_indices are the same as those used in BNBTree.
cd_object = cd(x, y, group_indices, Params)
sol, obj = cd_object.fit(lambda_0=10000, lambda_2=1000, warm_start=np.zeros(x.shape[1]))
print("Objective: ", obj)
print("Solution: ", sol)

Objective:  122612.01230645685
Solution:  [0.83003784 0.89108178 0.9187789  0.86131885 0.86513309 0.84007621
 0.85716887 0.91340719 0.83010533 0.83365805 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.85070453 0.87003903 0.78881707 0.80895292
 0.81106976 0.85479548 0.74801805 0.85210065 0.80881865 0.82579907
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.8602322  0.9234094
 0.92322945 0.90516406 0.93312893 0.93449248 0.8716701  0.89911364
 0.84711608 0.86712277 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.84593727 0.81365817 0.85040533 0.86648396 0.80078602 0.76979092
 0.8406259  0.84478778 0.86372381 0.86311515 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.79016068 0.83554466 0.8

In [5]:
"""This cell demonstrates how to run the CD + Local Search algorithm."""

from cd import cd_swaps

Params = {
    'NumIters': 1000, # maximum number of CD full cycles.
    'Tolerance': 1e-7 , # convergence tolerance.
    'Logging': False, # prints output while running
    'FeaturesOrder': 'Cyclic', # order in which CD updates the variables.
}

cd_swaps_object = cd_swaps(x, y, group_indices, Params)
# The fit function takes the same arguments as that of CD, in addition to "max_swaps" 
# which sets the maximum number of group swaps the local search algorithms can perform.
sol, obj = cd_swaps_object.fit(lambda_0=10000, lambda_2=1000, max_swaps=10, warm_start=np.zeros(x.shape[1]))
print("Objective: ", obj)
print("Solution: ", sol)

Objective:  122612.01230645685
Solution:  [0.83003784 0.89108178 0.9187789  0.86131885 0.86513309 0.84007621
 0.85716887 0.91340719 0.83010533 0.83365805 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.85070453 0.87003903 0.78881707 0.80895292
 0.81106976 0.85479548 0.74801805 0.85210065 0.80881865 0.82579907
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.8602322  0.9234094
 0.92322945 0.90516406 0.93312893 0.93449248 0.8716701  0.89911364
 0.84711608 0.86712277 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.84593727 0.81365817 0.85040533 0.86648396 0.80078602 0.76979092
 0.8406259  0.84478778 0.86372381 0.86311515 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.79016068 0.83554466 0.8